In [1]:
from numpy.random import randint
from numpy.random import rand
from collections import namedtuple
from random import *
from datetime import datetime
import itertools
import numpy as np
import matplotlib.pyplot as plt

# identifying the structure of Item (Object) and Candidate (A possible solution) 
Item = namedtuple("Item", ['id', 'size'])
Candidate = namedtuple("Candidate", ['items', 'fitness'])

In [2]:
# The function cost returns the cost of a collection of bins that is represented by these bins length
def cost(bins):
    return len(bins)
#________________________________________The class Bin________________________________________________________
class Bin(object):
    count = itertools.count()
    
    #This procedure simply create a new bin
    #The items contained on a bin are defined by their index in its items table 
    def __init__(self, capacity):
        self.id = next(Bin.count)
        self.capacity = capacity
        self.free_space = capacity
        self.items = []
        self.used_space = 0
    
    #This procedure adds a new item to a specific bin and update the values of used and free spaces
    def add_item(self, item):
        self.items.append(item)
        self.free_space -= item.size
        self.used_space += item.size
    
    #This procedure removes a specific item in a specific bin and update the values of used and free spaces
    def remove_item(self, item_index):
        item_to_remove = self.items[item_index]
        del self.items[item_index]
        self.free_space += item_to_remove.size
        self.used_space -= item_to_remove.size

    
    #This function allows us to know if we can add a specific item to a  specific bin or not
    def fits(self, item):
        return self.free_space >= item.size

    
    #This function display the content of a specific bin 
    def __str__(self):
        items = [str(it) for it in self.items]
        items_string = '[' + ' '.join(items) + ']'
        return "Bin n° " + str(self.id) + " containing the " + \
               str(len(self.items)) + " following items : " + items_string + \
               " with " + str(self.free_space) + " free space."
    
    #This function copy a bin into a new one with keeping the old bin
    def __copy__(self):
        new_bin = Bin(self.capacity)
        new_bin.free_space = self.free_space
        new_bin.used_space = self.used_space
        new_bin.items = self.items[:]
        return new_bin
        

In [3]:
#Now we will be introducing the different procedures and functions
#that allows us to build our Genetic Algorithm : 
# ----> Different Heuritic methods to help us define the fitness function
# ----> Population generator
# ----> Parents selection algorithms
# ----> Crossover algorithm
# ----> Mutation algorithm
#___________________________________________Heuristics_________________________________________________

def firstfit(items, current_bins, capacity):
    bins = [copy.copy(b) for b in current_bins]
    if not bins:
        bins = [Bin(capacity)]
    for item in items:
        if item.size > capacity:
            continue
        first_bin = next((bin for bin in bins if bin.free_space >= item.size), None)
        if first_bin is None:
            bin = Bin(capacity)
            bin.add_item(item)
            bins.append(bin)
        else:
            first_bin.add_item(item)
    return bins

In [4]:
#_______________________________Population Generator____________________________
# This function generates the initial population randomly 
def population_generator(items, capacity, population_size, greedy_solver):
    candidate = Candidate(items[:], fitness(items, capacity, greedy_solver))
    population = [candidate]
    #print(len(population[0].fitness))
    new_items = items[:]
    for i in range(population_size - 1):
        shuffle(new_items)
        candidate = Candidate(new_items[:], fitness(new_items, capacity, greedy_solver))
        if candidate not in population:
            population.append(candidate)
    return population


In [5]:
#____________________________________fitness___________________________
def fitness(candidate, capacity, greedy_solver):
    if greedy_solver == 'FF':
        return firstfit(candidate,[], capacity)

In [6]:
#________________________________Selection Methods__________________
# In K-Way tournament selection, we select K individuals from the population 
# at random (using a geometric function) 
# and select the best out of these to become a parent with a trournament_selection probability. 
# The same process is repeated for selecting the next parent.
def tournament_selection(population, tournament_selection_probability, k):
    candidates = [population[(randint(0, len(population) - 1))]]
    while len(candidates) < k:
        new_indiv = population[(randint(0, len(population) - 1))]
        if new_indiv not in candidates:
            candidates.append(new_indiv)
    ind = int(np.random.geometric(tournament_selection_probability, 1))
    while ind >= k:
        ind = int(np.random.geometric(tournament_selection_probability, 1))
    return candidates[ind]

In [7]:
def roulette_wheel_selection(population):
    max = sum([len(e.fitness) for e in population])
    pick = uniform(0, max)
    current = max
    for item in population:
        current -= len(item.fitness)
        if current < pick:
            return item

In [8]:
def SUS(population, n):
    selected = []
    pointers = []
    max = sum([len(e.fitness) for e in population])
    distance = max / n
    start = uniform(0, distance)
    for i in range(n):
        pointers.append(start + i * distance)
    for pointer in pointers:
        current = 0
        for item in population:
            current += len(item.fitness)
            if current > pointer:
                selected.append(item)
    return selected

In [9]:
def rank_selection(population):
    length = len(population)
    rank_sum = length * (length + 1) / 2
    pick = uniform(0, rank_sum)
    current = 0
    i = length
    for item in population:
        current += i
        if current > pick:
            return item
        i -= 1

In [10]:
#______________________________Crossover____________________________
def crossover(parent1, parent2):
    taken = [False] * len(parent1)
    child = []
    i = 0
    while i < len(parent1):
        element = parent1[i]
        if not taken[element.id]:
            child.append(element)
            taken[element.id] = True
        element = parent2[i]
        if not taken[element.id]:
            child.append(element)
            taken[element.id] = True
        i += 1
    return child 

In [11]:
#______________________________________Mutation_____________________
def mutation(member, capacity, greedy_solver):
    member_items = member.items
    a = randint(0, len(member_items) - 1)
    b = randint(0, len(member_items) - 1)
    while a == b:
        b = randint(0, len(member_items) - 1)
    c = member_items[a]
    member_items[a] = member_items[b]
    member_items[b] = c
    member = Candidate(member_items, fitness(member_items, capacity, greedy_solver))
    return member

In [12]:
def readFromFile(file):
    fichier = open(file, "r")
    w=[]
    i=0
    for line in fichier:
        if i==0:
            n=int(line)
            i=i+1  
        else:
            if i==1:
                c=int(line)
                i=i+1
            else: 
                if i>1:
                    w.append(int(line.rstrip('\n\r')))
                    i=i+1
    fichier.close()
    return n,c,w

In [13]:
#______________________________________Genetic Algorithm________________
def genetic_algorithm(weights, capacity, population_size, generations, max_no_change,k, tournament_selection_probability, crossover_probability, mutation_probability, greedy_solver, allow_duplicate_parents, selection_method):

    items = [Item]
    items = [ Item(i,weights[i]) for i in range(len(weights))]
    population = population_generator(items, capacity, population_size, greedy_solver)
    best_solution = fitness(items, capacity, greedy_solver)
    i = 0
    current_iteration = 0
    num_no_change = 0
    while current_iteration < generations and num_no_change < max_no_change:
        new_generation = []
        best_child = best_solution
        for j in range(population_size):


            if selection_method == 'SUS':
                first_parent = SUS(population, 1)[0].items
                second_parent = SUS(population, 1)[0].items
                if not allow_duplicate_parents:
                    while first_parent == second_parent:
                        second_parent = SUS(population, 1)[0].items
            elif selection_method == 'TS':
                first_parent = tournament_selection(population, tournament_selection_probability, k).items
                second_parent = tournament_selection(population, tournament_selection_probability, k).items
                if not allow_duplicate_parents:
                    while first_parent == second_parent:
                        second_parent = tournament_selection(population, tournament_selection_probability, k).items
            elif selection_method == 'RW':
                first_parent = roulette_wheel_selection(population).items
                second_parent = roulette_wheel_selection(population).items
                if not allow_duplicate_parents:
                    while first_parent == second_parent:
                        second_parent = roulette_wheel_selection(population).items
            elif selection_method == 'RS':
                first_parent = rank_selection(population).items
                second_parent = rank_selection(population).items
                if not allow_duplicate_parents:
                    while first_parent == second_parent:
                        second_parent = rank_selection(population).items
            else:
                return

            child = crossover(first_parent, second_parent)
            child = Candidate(child[:], fitness(child, capacity, greedy_solver))

            prob = random()
            if prob <= mutation_probability:
                child = mutation(child, capacity, greedy_solver)

            if len(child.fitness) < len(best_child):
                best_child = child.fitness
            new_generation.append(child)

        num_no_change += 1
        if len(best_child) < len(best_solution):
            best_solution = best_child
            num_no_change = 0
        population = [Candidate(p.items[:], p.fitness) for p in new_generation]
        population.sort(key=lambda candidate: len(candidate.fitness), reverse=True)
        #if i==30:
         #   print([len(population[l].fitness) for l in range(len(population))])
        current_iteration += 1
    #for d in range(len(best_solution)):
     #   print(best_solution[d].__str__())
    print("iterations number ",current_iteration,"stagnation: ", num_no_change)
    return len(best_solution)


POPULATION_SIZE = 50
MAX_GENERATIONS = 250
MAX_NO_CHANGE = 50
TOURNAMENT_SIZE = 20 
TOURNAMENT_SELECTION_RATE = 0.7
MUTATION_RATE = 0.3 
CROSSOVER_RATE = 0.6 


#weights = [49, 41, 34, 33, 29, 26, 26, 22, 20, 19, 40, 21, 45, 15, 18, 23, 43, 30]
#capacity = 100
file=r'D:\esi\2SQ3\S2\OPT\projet\Scholl\Scholl_1\N1C1W1_R.txt'
n, capacity, weights=readFromFile(file)
solution = genetic_algorithm(weights, capacity, POPULATION_SIZE, MAX_GENERATIONS, MAX_NO_CHANGE,TOURNAMENT_SIZE, TOURNAMENT_SELECTION_RATE, CROSSOVER_RATE, MUTATION_RATE, 'FF',False, 'RW')
print(solution)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\esi\\2SQ3\\S2\\OPT\\projet\\Scholl\\Scholl_1\\N1C1W1_R.txt'